In [1]:
from utils.octree import OCTree
from utils.mesh import TriMesh
from utils.correspondences import Correspondences
from sklearn.mixture import GaussianMixture

import numpy as np
import os
from PIL import Image
from tqdm import tqdm

import xml.dom.minidom

In [2]:
class LdmkProjector:
	def __init__(self):
		self.cor_builder = Correspondences()
		
	def solve_eq_size2(self, coef, dest):
		d = coef[0, 0] * coef[1, 1] - coef[1, 0] * coef[0, 1]
		dx = dest[:, 0] * coef[1, 1] - dest[:, 1] * coef[0, 1]
		dy = dest[:, 1] * coef[0, 0] - dest[:, 0] * coef[1, 0]
		return dx / d, dy / d 

	def inverse_project(self, scan, scan_octree, lmk, cam_mat, trans_mat):
		# import ipdb; ipdb.set_trace()
		# inv = np.linalg.inv(cam_mat)
		# lmk_mine = np.array([lmk[0, 0],lmk[0, 1], 1])
		# result = inv.dot(lmk_mine)
		# # result 添加一个0
		# result = np.insert(result,3,0)

		lmk_3d_x, lmk_3d_y = self.solve_eq_size2(cam_mat[:2, :2], np.column_stack((lmk[:, 0] - cam_mat[0, 2], lmk[:, 1] - cam_mat[1, 2])))
		lmk_3d_norm =  np.column_stack((lmk_3d_x, lmk_3d_y, np.ones(len(lmk))))

		tm_inv = np.linalg.inv(trans_mat)

		# rotate_result = tm_inv.dot(result)

		lmk_3d_norm = np.hstack((lmk_3d_norm, np.ones((len(lmk_3d_norm), 1)))).dot(tm_inv.T)[:, :3]
		ray_start = np.tile(tm_inv[:3, 3], len(lmk_3d_x)).reshape((-1, 3))
		ray_dir = lmk_3d_norm - ray_start
		ray_dir = ray_dir / (np.sqrt(np.sum(ray_dir ** 2, axis = 1))[:, np.newaxis] + 1e-12)

		ray_mesh = TriMesh()
		ray_mesh.vertices = ray_start
		ray_mesh.vert_normal = ray_dir

		ray_ind, tgt_face_ind, weights = self.cor_builder.nearest_tri_normal(ray_mesh, scan, scan_octree, dist_threshold = np.inf - 10, normal_threshold = -1)
		
		return ray_ind, tgt_face_ind, weights

	def gmm_fit(self, verts, num = 1):
		if len(verts.flatten()) <= 3:
			return verts
		gmm = GaussianMixture(n_components = num, random_state = 0).fit(verts)
		ind = np.argmax(gmm.weights_)
		return gmm.means_[ind]

	def ldmk_3d_detect(self, scan, lmk2d_lst, cam_mat_lst, trans_mat_lst):
		octree = OCTree()
		octree.from_triangles(scan.vertices, scan.faces, np.arange(scan.face_num()))
		scan.cal_face_normal()

		view_num = len(lmk2d_lst)
		ldmk_num = lmk2d_lst[0].shape[0]

		lmk3d_lst = np.zeros((view_num, ldmk_num, 3), dtype = np.float32)
		normal_lst = np.zeros((view_num, ldmk_num, 3), dtype = np.float32)
		lmk3d_mask = np.full((view_num, ldmk_num), False)

		for i in range(len(lmk2d_lst)):
			src_ind, tgt_face_ind, weights = self.inverse_project(scan, octree, lmk2d_lst[i], cam_mat_lst[i], trans_mat_lst[i])
			lmk3d_lst[i, src_ind, :] = np.sum(scan.vertices[scan.faces[tgt_face_ind]] * weights[:, :, np.newaxis], axis = 1)
			normal_lst[i, src_ind, :] = scan.face_normal[tgt_face_ind]
			lmk3d_mask[i, src_ind] = True 

		lmk3d = np.zeros((ldmk_num, 3), dtype = np.float32)

		for i in range(ldmk_num):
			lmk3d_i = lmk3d_lst[:, i, :]
			mask_i = lmk3d_mask[:, i]
			lmk3d_i = lmk3d_i[mask_i]

			ray_mesh = TriMesh()
			ray_mesh.vertices = self.gmm_fit(lmk3d_i).reshape((-1, 3))
			ray_mesh.vert_normal = np.mean(normal_lst[:, i, :], axis = 0).reshape((-1, 3))
			src_ind, tgt_face_ind, weights = self.cor_builder.nearest_tri_normal(ray_mesh, scan, octree, dist_threshold = np.inf - 10, normal_threshold = -1)

			if len(src_ind) == 0:
				print(f'3d landmark err!: {i}')
			if len(src_ind) != 0:
				lmk3d[i] = np.sum(scan.vertices[scan.faces[tgt_face_ind]] * weights[:, :, np.newaxis], axis = 1).flatten()
		
		return lmk3d

In [3]:
import numpy as np 
import xml.dom.minidom
from PIL import Image

def parse_cam_param(xml_path, cam_idx_lst):
	root = xml.dom.minidom.parse(xml_path)
	cam_params = root.getElementsByTagName('camera')
	sensor_params = root.getElementsByTagName('sensor')

	cam_mat_lst = []
	trans_mat_lst = []
	valid_lst = []

	for cam_idx in cam_idx_lst:
		if '.jpg' in cam_idx or '.JPG' in cam_idx or '.png' in cam_idx or '.PNG' in cam_idx:
			cam_idx = cam_idx[:-4]
		cam_param = None
		sensor_param = None

		for p in cam_params:
			if p.getAttribute('label') == cam_idx:
				cam_param = p
				break

		sensor_id = cam_param.getAttribute('sensor_id')
		for p in sensor_params:
			if p.getAttribute('id') == sensor_id:
				sensor_param = p 
				break

		width = int(sensor_param.getElementsByTagName('resolution')[0].getAttribute('width'))
		height = int(sensor_param.getElementsByTagName('resolution')[0].getAttribute('height'))
	
		f = float(sensor_param.getElementsByTagName('f')[0].firstChild.data)
		cx = float(sensor_param.getElementsByTagName('cx')[0].firstChild.data)
		cy = float(sensor_param.getElementsByTagName('cy')[0].firstChild.data)

		cam_mat = np.zeros((3, 3))
		cam_mat[2, 2] = 1.0

		# if width > height:
		# 	cam_mat[0, 1] = -f 
		# 	cam_mat[1, 0] = f 
		# 	cam_mat[0, 2] = height / 2 - cy
		# 	cam_mat[1, 2] = width / 2 + cx
		# else:
		cam_mat[0, 0] = f 
		cam_mat[1, 1] = f 
		cam_mat[0, 2] = width / 2 + cx 
		cam_mat[1, 2] = height / 2 + cy

		if len(cam_param.getElementsByTagName('transform')) == 0:
			valid_lst.append(False)
			continue

		transform = cam_param.getElementsByTagName('transform')[0].firstChild.data
		ss = transform.split(' ')
		trans_mat = np.zeros((4, 4))
		for i in range(4):
			for j in range(4):
				trans_mat[i, j] = float(ss[i * 4 + j])

		trans_mat = np.linalg.inv(trans_mat)

		cam_mat_lst.append(cam_mat) 
		trans_mat_lst.append(trans_mat)
		valid_lst.append(True)
	
	return cam_mat_lst, trans_mat_lst, valid_lst

def gmm_fit(verts, num = 1):
    if len(verts.flatten()) <= 3:
        return verts[0]
    gmm = GaussianMixture(n_components = num, random_state = 0).fit(verts)
    ind = np.argmax(gmm.weights_)
    return gmm.means_[ind]

In [4]:
import cv2
import cv2.aruco as aruco
import numpy as np

# Define parameters for the CharucoBoard
num_squares_x = 7
num_squares_y = 10
square_length = 0.04  # length of each square side in meters
marker_length = 0.02  # length of the markers in meters
dictionary = aruco.getPredefinedDictionary(aruco.DICT_5X5_1000)  # you can choose a different dictionary

# Define a nonzero start ID for aruco markers
start_id = 200

# Create CharucoBoard with a nonzero start ID
board1 = aruco.CharucoBoard(
    (num_squares_x, num_squares_y),
    squareLength=square_length,
    markerLength=marker_length,
    dictionary=dictionary,
    ids=np.arange(start_id, start_id+num_squares_x*num_squares_y//2, dtype=np.int32)
)

board2 = aruco.CharucoBoard(
    (num_squares_x, num_squares_y),
    squareLength=square_length,
    markerLength=marker_length,
    dictionary=dictionary,
    ids=board1.getIds() + len(board1.getIds()),
)


In [5]:
def get_board_points_mean_pos_dict(board, dictionary, mesh_path, image_dir,cameras_path):

    img_names = sorted([os.path.splitext(filename)[0] for filename in os.listdir(image_dir)])

    cam_mat_lst, trans_mat_lst, valid_lst = parse_cam_param(cameras_path, img_names)

    res_list = []
    res_ids = None
    valid_cam_lst = []
    valid_trans_lst = []
    for i, img_name  in enumerate(img_names):
        img_path = image_dir + "/" +img_name + ".png"
        img = cv2.imread(img_path)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        markerCorners1, markerIds1, rejectedImgPoints1 = cv2.aruco.detectMarkers(img, dictionary)
        if len(markerCorners1) == 0:
            continue
        retval, charuco_corners, charuco_ids = aruco.interpolateCornersCharuco(markerCorners1, markerIds1, gray, board)
        if retval == 0:
            continue
        res_list.append(charuco_corners[:,0,:])
        if res_ids is None:
            res_ids = charuco_ids
        else:
            res_ids = np.vstack((res_ids,charuco_ids))
        valid_cam_lst.append(cam_mat_lst[i])
        valid_trans_lst.append(trans_mat_lst[i])
    mesh = TriMesh()
    mesh.load(mesh_path)

    projector = LdmkProjector()
    for i in range(len(res_list)):
        lmk_3d = projector.ldmk_3d_detect(mesh, [res_list[i]], [valid_cam_lst[i]], [valid_trans_lst[i]])
        if i == 0 :
            lmk_3d_list = lmk_3d
        else:
            lmk_3d_list = np.vstack((lmk_3d_list,lmk_3d))
        
        # 创建一个空字典用于存储相同ID的点
    points_dict = {}

    # 遍历res_ids和lmk_3d_list，将相同ID的点组合成子列表
    for id, point in zip(res_ids, lmk_3d_list):
        if int(id) not in points_dict:
            points_dict[int(id)] = []  # 初始化空子列表
        points_dict[int(id)].append(point)  # 将点添加到相应的子列表中

    for id, points in points_dict.items():
        points_dict[id] = gmm_fit(np.array(points))

    return points_dict

    

In [6]:
exp = "align_merge_out_2023_12_07_90degree_ldr"
mesh_path ='/home/yuruihan/DS-FaceScape/hdr_emitter/metashape_output/{}/model.obj'.format(exp)
image_dir = "/home/yuruihan/DS-FaceScape/hdr_emitter/{}".format(exp)
cameras_path = '/home/yuruihan/DS-FaceScape/hdr_emitter/metashape_output/{}/camera.xml'.format(exp)
board1_points_dict = get_board_points_mean_pos_dict(board1, dictionary, mesh_path, image_dir, cameras_path)
board2_points_dict = get_board_points_mean_pos_dict(board2, dictionary, mesh_path, image_dir, cameras_path)

In [27]:
board1_points_dict

{36: array([ 0.09233526, -0.07829256, -3.23631636]),
 37: array([ 0.04748182, -0.08115034, -3.239055  ]),
 42: array([ 0.09450176, -0.0744091 , -3.28042277]),
 43: array([ 0.05019902, -0.07763858, -3.28322041]),
 49: array([ 0.05284342, -0.07421737, -3.32730993]),
 25: array([ 0.04165594, -0.08828286, -3.15052001]),
 26: array([-2.19326457e-03, -9.14384499e-02, -3.15306282e+00]),
 27: array([-0.04617888, -0.09412451, -3.15637803]),
 32: array([ 9.46025102e-05, -8.79082959e-02, -3.19740522e+00]),
 33: array([-0.04342739, -0.09060444, -3.2003684 ]),
 38: array([ 2.41955498e-03, -8.42190906e-02, -3.24192226e+00]),
 5: array([-0.14495456, -0.11373308, -2.98384146]),
 15: array([-0.05194   , -0.10119323, -3.0678022 ], dtype=float32),
 31: array([ 0.04456473, -0.08462416, -3.19450863]),
 0: array([ 0.07621915, -0.09940865, -2.97025879]),
 1: array([ 0.03146271, -0.10244664, -2.9729017 ]),
 2: array([-0.01296286, -0.10519101, -2.97561455]),
 3: array([-0.05701434, -0.10822371, -2.9780722 ], d

In [28]:
board2_points_dict

{49: array([-0.14643307, -0.16984299, -3.68430467]),
 0: array([ 0.22215854, -0.14320473, -3.69733634]),
 1: array([ 0.21777891, -0.14683727, -3.65314326]),
 5: array([ 0.20080774, -0.16070747, -3.4764272 ]),
 6: array([ 0.1770762 , -0.14551627, -3.70185447]),
 7: array([ 0.1726314 , -0.14896523, -3.65706865]),
 11: array([ 0.15662052, -0.16306787, -3.47967253]),
 17: array([ 0.11178491, -0.16599118, -3.4852668 ]),
 30: array([-0.00775441, -0.15765121, -3.71828547]),
 36: array([-0.05373737, -0.16030059, -3.72333537]),
 37: array([-0.05564744, -0.16393111, -3.6771399 ]),
 47: array([-0.11384159, -0.18018579, -3.50336963]),
 42: array([-0.09884489, -0.16354415, -3.72602777]),
 41: array([-0.07195692, -0.17763098, -3.49874073]),
 40: array([-0.06750536, -0.17418311, -3.54271468]),
 43: array([-0.10328758, -0.16720642, -3.67885208]),
 48: array([-0.14280629, -0.16637916, -3.72843182]),
 2: array([ 0.21351021, -0.15037071, -3.60784101]),
 3: array([ 0.20954051, -0.15360705, -3.56312954]),


In [30]:
# 取出两个字典中相同ID的点，形成两个新列表，一一对应
board1_points = []
board2_points = []
for id in board1_points_dict:
    if id in board2_points_dict:
        board1_points.append(board1_points_dict[id])
        board2_points.append(board2_points_dict[id])

# 将两个列表转换为数组
board1_points = np.array(board1_points)
board2_points = np.array(board2_points)


In [31]:
board1_points

array([[ 0.09233526, -0.07829256, -3.23631636],
       [ 0.04748182, -0.08115034, -3.239055  ],
       [ 0.09450176, -0.0744091 , -3.28042277],
       [ 0.05019902, -0.07763858, -3.28322041],
       [ 0.05284342, -0.07421737, -3.32730993],
       [-0.14495456, -0.11373308, -2.98384146],
       [ 0.07621915, -0.09940865, -2.97025879],
       [ 0.03146271, -0.10244664, -2.9729017 ],
       [-0.01296286, -0.10519101, -2.97561455],
       [-0.05701434, -0.10822371, -2.97807217],
       [-0.10149025, -0.11115269, -2.98062015],
       [-0.03863626, -0.08338223, -3.28857332],
       [-0.08322868, -0.08620117, -3.29119786],
       [-0.12781671, -0.08877207, -3.29374385],
       [-0.03638505, -0.07936872, -3.33286601],
       [-0.08062988, -0.08272211, -3.33565056],
       [-0.12447085, -0.08532991, -3.33810226],
       [ 0.09692845, -0.07131181, -3.32431889],
       [ 0.00554239, -0.08043469, -3.28574772],
       [ 0.00788388, -0.07698214, -3.33022738],
       [-0.08457316, -0.08968747, -3.247

In [32]:
board2_points

array([[-0.05373737, -0.16030059, -3.72333537],
       [-0.05564744, -0.16393111, -3.6771399 ],
       [-0.09884489, -0.16354415, -3.72602777],
       [-0.10328758, -0.16720642, -3.67885208],
       [-0.14643307, -0.16984299, -3.68430467],
       [ 0.20080774, -0.16070747, -3.4764272 ],
       [ 0.22215854, -0.14320473, -3.69733634],
       [ 0.21777891, -0.14683727, -3.65314326],
       [ 0.21351021, -0.15037071, -3.60784101],
       [ 0.20954051, -0.15360705, -3.56312954],
       [ 0.20553513, -0.1571324 , -3.51964521],
       [-0.10850029, -0.17382422, -3.58959627],
       [-0.1119403 , -0.17710903, -3.54648987],
       [-0.11384159, -0.18018579, -3.50336963],
       [-0.15370244, -0.17671275, -3.59303308],
       [-0.15712902, -0.18011576, -3.54957318],
       [-0.15885802, -0.18335177, -3.5057407 ],
       [-0.14280629, -0.16637916, -3.72843182],
       [-0.10521689, -0.17042643, -3.63368654],
       [-0.15038079, -0.1733461 , -3.63718271],
       [-0.06750536, -0.17418311, -3.542

In [33]:
import estimate_rigid_transform as ert
M = ert.affine_matrix_from_points(board1_points.transpose(), board2_points.transpose(), scale=True)

In [34]:
M

array([[ 2.59342657e-02, -1.45655710e-01,  1.01101263e+00,
         3.20548323e+00],
       [ 4.00010563e-03,  1.01134517e+00,  1.45601009e-01,
         3.89452934e-01],
       [-1.02144316e+00,  2.62387214e-04,  2.62396291e-02,
        -3.54361802e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00]])

In [36]:
# cv2 检测marker
res_list = []

test_img_name = "IMG_0309"

test_img_path = os.path.join(image_dir, test_img_name + ".png")
test_img = cv2.imread(test_img_path)
test_gray = cv2.cvtColor(test_img, cv2.COLOR_BGR2GRAY)
test_markerCorners1, test_markerIds1, rejectedImgPoints1 = cv2.aruco.detectMarkers(test_img, dictionary)
retval, test_charuco_corners1, test_charuco_ids1 = aruco.interpolateCornersCharuco(test_markerCorners1, test_markerIds1, test_gray, board1)
retval, test_charuco_corners2, test_charuco_ids2 = aruco.interpolateCornersCharuco(test_markerCorners1, test_markerIds1, test_gray, board2)

In [37]:
# find the same id in board1 and board2
test_board1_points = []
test_board2_points = []
for id in test_charuco_ids1:
    if id in test_charuco_ids2:
        # get the index of the same id
        index1 = np.argwhere(test_charuco_ids1 == id)[0][0]
        index2 = np.argwhere(test_charuco_ids2 == id)[0][0]
        # get the points of the same id
        test_board1_points.append(test_charuco_corners1[index1])
        test_board2_points.append(test_charuco_corners2[index2])

In [38]:
# test_board1_points
# test_board2_points 在原图上用半径为10的圆标注出来 空心圆
# 在图像上绘制圆
marked_img = test_img.copy()
for point in test_board1_points:
    cv2.circle(marked_img, (int(point[0][0]), int(point[0][1])), 10, (0, 0, 255), 2)
for point in test_board2_points:
    cv2.circle(marked_img, (int(point[0][0]), int(point[0][1])), 10, (0, 0, 255), 2)


In [39]:
cv2.imwrite("./{}_mark.png".format(test_img_name), marked_img)

True

In [40]:
test_img_name

'IMG_0309'

In [ ]:
cam_mat_lst, trans_mat_lst, valid_lst = parse_cam_param(cameras_path, [test_img_name])
def p2d2pd3(board_points, mesh_path, cam_mat_lst, trans_mat_lst): 
    res_list = np.array(board_points)
    res_list = res_list[:,0,:]

    mesh = TriMesh()
    mesh.load(mesh_path)

    projector = LdmkProjector()
    for i in range(len([res_list])):
        lmk_3d = projector.ldmk_3d_detect(mesh, [res_list], [cam_mat_lst[i]], [trans_mat_lst[i]])
        if i == 0 :
            lmk_3d_list = lmk_3d
        else:
            lmk_3d_list = np.vstack((lmk_3d_list,lmk_3d))
    return lmk_3d_list
board1_lmk_3d_list = p2d2pd3(test_board1_points,mesh_path, cam_mat_lst, trans_mat_lst)

In [ ]:
# lmk_3d_list  M transform
transformed_lmk_3d_list = []
for point_3d in board1_lmk_3d_list:
    point_3d = np.append(point_3d,1)
    point_3d = np.dot(M,point_3d)
    point_3d = point_3d[:3]
    transformed_lmk_3d_list.append(point_3d)
transformed_lmk_3d_list = np.array(transformed_lmk_3d_list)


In [ ]:
transformed_lmk_3d_list

In [ ]:
board2_lmk_3d_list = p2d2pd3(test_board2_points,mesh_path, cam_mat_lst, trans_mat_lst)

In [ ]:
board2_lmk_3d_list

In [ ]:
trans_mat_lst[0]

In [ ]:
def transform_mat2vec(trans_mat):
        # 提取旋转部分
    R = trans_mat[:3, :3]

    # 提取平移部分
    t = trans_mat[:3, 3]

    # 将旋转矩阵转换为旋转向量
    rvec, _ = cv2.Rodrigues(R)

    # 输出旋转向量和平移向量
    print("旋转向量 (rvec):")
    print(rvec)

    print("平移向量 (tvec):")
    print(t)
    return rvec,t

In [ ]:
rvec,tvec = transform_mat2vec(trans_mat_lst[0])
board1_transformed_points_2d, _ = cv2.projectPoints(transformed_lmk_3d_list, rvec,tvec, cam_mat_lst[0], distCoeffs=None)

In [ ]:
board1_transformed_points_2d

In [ ]:
test_board2_points

In [ ]:
# 在marked_img上绘制board1_transformed_points_2d
for point in board1_transformed_points_2d:
    cv2.circle(marked_img, (int(point[0][0]), int(point[0][1])), 10, (255, 0, 0), 2)
cv2.imwrite("./{}_mark.png".format(test_img_name), marked_img)